In [1]:
# %%
import numpy as np
import sapien.core as sapien
from PIL import Image
import json
from datagen.data_utils import *
from pathlib import Path as P
from utils.visualization import overlay_images
# from utils.viewpoint import calculate_E2 as CE2
# from utils.viewpoint import conversion_matrix_axis as cma
from utils.viewpoint import * 
# from utils.viewpoint import conversion_matrix
# %%
# def setup_scene(urdf):
#     # Set up the SAPIEN scene and camera
#     camera, asset, scene = scene_setup(urdf_file=urdf)
#     return camera, asset, scene
# conversion_matrix = np.array([
#     [0, 0, -1],
#     [-1, 0, 0],
#     [0, 1, 0]
# # ])
# def normalize(v):
#     return v / np.sqrt(np.sum(v**2))

# def calculate_E2(E1, axis_position, axis_direction, angle_degrees):
#     # Convert the angle from degrees to radians
#     angle_radians = degrees_to_radians(angle_degrees)
    
#     # Create a 3x3 rotation matrix around the axis
#     R = get_rotation_axis_angle(axis_direction, angle_radians)
#     # Create a 4x4 transformation matrix for the rotation
#     rotation_matrix = np.eye(4)
#     rotation_matrix[:3, :3] = R
#     print(R)
    
#     # Create a 4x4 transformation matrix for translation to the axis position
#     translation_matrix = np.eye(4)
#     translation_matrix[:3, 3] = axis_position
#     # Calculate E2 by combining translation and rotation with E1
#     # E2 = np.dot(rotation_matrix, np.dot(rotation_matrix, E1))
#     E2 = change_apply_change_basis(E1, rotation_matrix, translation_matrix)
#     return E2

# def change_apply_change_basis(A, T, P):
#     """
#     Perform change of basis, apply transformation, and change back to the original basis.

#     Args:
#     A (numpy.ndarray): The original matrix in the B1 basis.
#     T (numpy.ndarray): The transformation matrix to be applied.
#     P (numpy.ndarray): The change of basis matrix from B1 to B2.

#     Returns:
#     numpy.ndarray: The resulting matrix after the entire process, expressed in the B1 basis.
#     """
#     # Step 1: Change A to the B2 basis
#     A_B2 = np.linalg.inv(P).dot(A).dot(P)
    
#     # Step 2: Apply the transformation T in the B2 basis
#     A_B2_transformed = T.dot(A_B2)
    
#     # Step 3: Change A_B2_transformed back to the original B1 basis
#     A_B1_transformed = P.dot(A_B2_transformed).dot(np.linalg.inv(P))
    
#     return A_B1_transformed

# def get_rotation_axis_angle(k, theta):
#     '''
#     Rodrigues' rotation formula
#     args:
#     * k: direction vector of the axis to rotate about
#     * theta: the (radian) angle to rotate with
#     return:
#     * 3x3 rotation matrix
#     '''
#     k = normalize(k)
#     kx, ky, kz = k[0], k[1], k[2]
#     cos, sin = np.cos(theta), np.sin(theta)
#     R = np.zeros((3, 3))
#     R[0, 0] = cos + (kx**2) * (1 - cos)
#     R[0, 1] = kx * ky * (1 - cos) - kz * sin
#     R[0, 2] = kx * kz * (1 - cos) + ky * sin
#     R[1, 0] = kx * ky * (1 - cos) + kz * sin
#     R[1, 1] = cos + (ky**2) * (1 - cos)
#     R[1, 2] = ky * kz * (1 - cos) - kx * sin
#     R[2, 0] = kx * kz * (1 - cos) - ky * sin
#     R[2, 1] = ky * kz * (1 - cos) + kx * sin
#     R[2, 2] = cos + (kz**2) * (1 - cos)
#     return R


# def transform_V1_to_V2(joint_state, V1, delta_rotation):
#     try:
#         # Extract the rotation matrix R_joint from the joint state
#         R_joint = joint_state[:3, :3]

#         u = R_joint[0, 2]
#         v = R_joint[1, 2]
#         w = R_joint[2, 2]


#         # Calculate the rotation matrix R_theta for the delta rotation along the joint axis
#         cos_theta = np.cos(delta_rotation)
#         sin_theta = np.sin(delta_rotation)
#         R_theta = np.array([[cos_theta + u**2 * (1 - cos_theta), u * v * (1 - cos_theta) - w * sin_theta, u * w * (1 - cos_theta) + v * sin_theta],
#                             [v * u * (1 - cos_theta) + w * sin_theta, cos_theta + v**2 * (1 - cos_theta), v * w * (1 - cos_theta) - u * sin_theta],
#                             [w * u * (1 - cos_theta) - v * sin_theta, w * v * (1 - cos_theta) + u * sin_theta, cos_theta + w**2 * (1 - cos_theta)]])


#         # Create 4x4 transformation matrices
#         T_joint = np.eye(4)
#         T_joint[:3, :3] = R_joint

#         T_theta = np.eye(4)
#         T_theta[:3, :3] = R_theta

#         # Compute V2 by multiplying the transformation matrices
#         # V2 = np.dot(np.dot(np.dot(T_joint, T_theta), np.linalg.inv(T_joint)), V1)
#         V2_prime = np.dot(np.linalg.inv(T_joint), np.dot(T_theta, V1))
#         V2 = np.dot(V2_prime, T_joint)

#         return V2

#     except Exception as e:
#         print(f"Error calculating V2: {str(e)}")
#         return None
# def degrees_to_radians(degrees):
#     radians = degrees * (math.pi / 180)
#     return radians
def render_image(camera, scene, save_path):
    # Render an image using the provided camera and save it to the specified path
    scene.step()
    scene.update_render()
    camera.take_picture()
    rgba = camera.get_float_texture('Color')
    rgba_img = (rgba * 255).clip(0, 255).astype("uint8")
    rgba_pil = Image.fromarray(rgba_img, 'RGBA')
    rgba_pil.save(save_path)

def load_json_to_dict(fname):
    # Load JSON data from a file into a dictionary
    try:
        with open(fname, 'r') as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"Error loading JSON from {fname}: {str(e)}")
        return None

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
urdf = 'data_base/laptop/10211/mobility.urdf'
# Load JSON data
json_fname = './data_base/laptop/10211/mobility_v2.json'
meta = load_json_to_dict(json_fname)
for link in meta:
    if link['joint'] == 'hinge':
        print(link['id'])
        print(link['jointData'])
        origin =  link['jointData']['axis']['origin']
        direction =  link['jointData']['axis']['direction']
# Setup the scene
camera, asset, scene = scene_setup(urdf)
# %%
# point = random_point_in_sphere(5, theta_range=[-0.01 * math.pi, 0.01*math.pi], phi_range=[0.01*math.pi, 0.01*math.pi])

def point_in_sphere(r, theta, phi):
    x = r * math.sin(phi) * math.cos(theta)
    y = r * math.sin(phi) * math.sin(theta)
    z = r * math.cos(phi)
    
    return x, y, z

point = point_in_sphere(5, 0.7*math.pi, degrees_to_radians(100))
mat44 = calculate_cam_ext(point)
print("point")
print(point)
print('mat44:')
print(mat44)
# print(point)
# mat44 = np.eye(4)
# mat44[0, -1] = 4
# mat44[1, -1] = 2
# mat44[2, -1] = 1
camera.set_pose(sapien.Pose.from_transformation_matrix(mat44))
# Render image with default pose
render_image(camera, scene, './draft/test_view.png')
seg_labels = camera.get_uint32_texture('Segmentation')  # [H, W, 4]
seg_view = seg_labels[..., 1].astype(np.uint8)  # actor-level
seg_pil = Image.fromarray(seg_view)
seg_pil.save('./draft/test_view_seg.png')
view_0 = camera.get_model_matrix()
art_degree = 10

# Rotate the joint and render a new image
asset.set_qpos(degrees_to_radians(art_degree))
render_image(camera, scene, './draft/test_view_15.png')
seg_labels = camera.get_uint32_texture('Segmentation')  # [H, W, 4]
seg_view_15 = seg_labels[..., 1].astype(np.uint8)  # actor-level
seg_view_15 = Image.fromarray(seg_view_15)
seg_view_15.save('./draft/test_view_15_seg.png')
# Calculate joint state and transformation
joint_state = asset.get_links()[-1].pose.to_transformation_matrix()
# v2 = transform_V1_to_V2(joint_state, mat44, degrees_to_radians(15))


# dirs, ori = np.dot(conversion_matrix, direction), np.dot(conversion_matrix, origin)


# v2 = calculate_E2(mat44, ori, dirs, art_degree)
v2 = calculate_E2(mat44, origin, direction, art_degree)
# view_2 = calculate_E2(view_0, np.array(origin), np.array(direction), art_degree)
# Set the new joint configuration and render the image
asset.set_qpos(0)
camera.set_pose(sapien.Pose.from_transformation_matrix(v2))
render_image(camera, scene, './draft/test_view_15_v2.png')
seg_labels = camera.get_uint32_texture('Segmentation')  # [H, W, 4]
seg_view_15_v2 = seg_labels[..., 1].astype(np.uint8)  # actor-level
seg_view_15_v2_pil = Image.fromarray(seg_view_15_v2)
seg_view_15_v2_pil.save('./draft/test_view_15_v2_seg.png')

view_1 = camera.get_model_matrix()


inerf_json = {
    'view_0': view_0.tolist(),
    'view_1': view_1.tolist()
}

with open('./draft/inerf_pose.json', 'w') as f:
    json.dump(inerf_json, f)


# Overlay images
rgba_15_pil = Image.open('./draft/test_view_15.png')
rgba_v2_pil = Image.open('./draft/test_view_15_v2.png')
overlayed = overlay_images(rgba_15_pil, rgba_v2_pil, 0.5)
overlayed.save('./draft/overlayed_view.png')


rgba_pil = Image.open('./draft/test_view.png')
overlayed = overlay_images(rgba_pil, rgba_15_pil, 0.5)
overlayed.save('./draft/overlayed_art.png')

overlayed = overlay_images(rgba_pil, rgba_v2_pil, 0.5)
overlayed.save('./draft/overlayed_base.png')

3
{'axis': {'origin': [0, -0.007706040424053753, -0.24714714808389615], 'direction': [1, 0, 0]}, 'limit': {'a': 105.11999999999999, 'b': -27, 'noLimit': False}}
point
(-2.8942773678193205, 3.9836310418954106, -0.8682408883346515)
mat44:
[[ 0.57885547  0.80901699 -0.10206784 -2.89427737]
 [-0.79672621  0.58778525  0.14048433  3.98363104]
 [ 0.17364818 -0.          0.98480775 -0.86824089]
 [ 0.          0.          0.          1.        ]]
[[ 0.98480775 -0.         -0.17364818]
 [ 0.          1.         -0.        ]
 [ 0.17364818  0.          0.98480775]]
Pose([0, 0, 0], [0.996195, 0, -0.0871557, 0])


In [17]:
view_0 # view
print(view2pose(view_0))

pose rotation
[[-5.78855515e-01 -8.09017062e-01 -1.02067828e-01]
 [ 7.96726227e-01 -5.87785244e-01  1.40484333e-01]
 [-1.73648238e-01  2.98023224e-08  9.84807730e-01]]
view
[[-8.0901706e-01 -1.0206783e-01 -5.7885551e-01 -2.8942773e+00]
 [-5.8778524e-01  1.4048433e-01  7.9672623e-01  3.9836311e+00]
 [ 2.9802322e-08  9.8480773e-01 -1.7364824e-01 -8.6824089e-01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
view2pose
[[-8.0901706e-01 -1.0206783e-01 -5.7885551e-01 -2.8942773e+00]
 [-5.8778524e-01  1.4048433e-01  7.9672623e-01  3.9836311e+00]
 [ 2.9802322e-08  9.8480773e-01 -1.7364824e-01 -8.6824089e-01]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
mannual
[[ 5.78855515e-01 -8.09017062e-01  1.02067828e-01 -2.89427733e+00]
 [ 7.96726227e-01  5.87785244e-01 -1.40484333e-01  3.98363113e+00]
 [-1.73648238e-01  2.98023224e-08  9.84807730e-01 -8.68240893e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [3]:
mat44 # pose
print(mat44)
print(view2pose(mat44))

[[ 0.57885547  0.80901699 -0.10206784 -2.89427737]
 [-0.79672621  0.58778525  0.14048433  3.98363104]
 [ 0.17364818 -0.          0.98480775 -0.86824089]
 [ 0.          0.          0.          1.        ]]
[[ 0.10206784 -0.57885547  0.80901699 -2.89427737]
 [-0.14048433  0.79672621  0.58778525  3.98363104]
 [-0.98480775 -0.17364818  0.         -0.86824089]
 [ 0.          0.          0.          1.        ]]


# transform test with numpy

In [14]:
diff = view_0 - pose2view(mat44)
diff[abs(diff) < 1e-7] = 0
print(diff)

diff = mat44 - view2pose(pose2view(mat44))
diff[abs(diff) < 1e-7] = 0
print(diff)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## transform test with torch

In [17]:
import torch

mat44_torch = torch.Tensor(mat44)
view_torch = torch.Tensor(view_0)
pose = sapien.Pose.from_transformation_matrix(mat44)
pose.q


array([ 0.887616  , -0.03956788, -0.07765634, -0.45226294], dtype=float32)

In [16]:
diff = view_torch - pose2view_torch(mat44_torch)
diff[abs(diff) < 1e-7] = 0
print(diff)

diff = mat44_torch - view2pose_torch(pose2view_torch(mat44_torch))
diff[abs(diff) < 1e-7] = 0
print(diff)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [21]:
ori, dirs = convert_ori_dir(origin, direction)

# Convert the angle from degrees to radians
angle_radians = degrees_to_radians(art_degree)

# Create a 3x3 rotation matrix around the axis
R = get_rotation_axis_angle(dirs, angle_radians)

from utils.rotation import *
Q = matrix_to_quaternion(torch.Tensor(R))

In [22]:
Q

tensor([ 0.9962,  0.0000, -0.0872,  0.0000])

In [23]:
test_mat44 = np.eye(4)
test_mat44[:3, :3] = R
test_pose = sapien.Pose.from_transformation_matrix(test_mat44)
test_pose.q

array([ 0.9961947 ,  0.        , -0.08715574,  0.        ], dtype=float32)

In [26]:
ori

array([ 0.24714715,  0.        , -0.00770604])